Your markdown or comments here

In [12]:
# -*- coding: utf-8 -*-
"""embedding-crawler.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1veoXGbqXGOxHJrYH6ZehSCyoLBhXvML9
"""

# for embedding.py file
# python3 -m venv venv
# source venv/bin/activate
# pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 SpeechRecognition opencv-python
# python3 embedding.py

%pip install --upgrade pip -q
%pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 -q
%pip install youtube-transcript-api bs4 pypdf -q
%pip install SpeechRecognition -q
%pip install opencv-python -q
%pip install beautifulsoup4 -q
%pip install python-pptx -q
%pip install 'unstructured[all]' -q

## For mac
# brew install libmagic
# brew install file

## For Ubuntu
# sudo apt-get update
# sudo apt-get install libmagic1 libmagic-dev


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:

# --- Step 1: 
from dotenv import load_dotenv
import os
import time
import requests
import yt_dlp
from bs4 import BeautifulSoup

from langchain_community.document_loaders import PyPDFLoader, YoutubeLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredPowerPointLoader


In [14]:
# --- Step 2: Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [15]:
# --- Step 3: Web crawler
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; FidesCrawler/1.0)"
}

web_urls = [
    "https://www.fidesinnova.io/",
    "https://fidesinnova.io/devices/",
    "https://fidesinnova.io/Contacts/",
    "https://fidesinnova.io/courses/",
    "https://linktr.ee/fidesinnova/",
    "https://play.google.com/store/apps/details?id=io.fidesinnova.front&pli=1/",
    "https://fidesinnova.io/blog-standard/",
    "https://fidesinnova.io/About/",
    "https://fidesinnova.io/#articles/",
    "https://discord.com/invite/NQdM6JGwcs/",
    "https://fidesinnova.io/service-market/",
    "https://fidesinnova.io/service-contract-2/",
    "https://fidesinnova.io/d2pos/",
    "https://fidesinnova.io/web3/",
    "https://fidesinnova.io/service-contract/",
    "https://fidesinnova.io/miotn/",
    "https://fidesinnova.io/consensus-algorithms/",
    "https://fidesinnova.io/mqtt-mqtts/",
    "https://explorer.fidesinnova.io/",
    "https://apps.apple.com/ca/app/fidesinnova/id6477492757/",
    "https://www.youtube.com/@FidesInnova/playlists/",
    "https://x.com/FidesInnova/"

]

def crawl_web_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else "No Title"
        text = soup.get_text(separator='\n', strip=True)
        return Document(page_content=text, metadata={"source": url, "title": title, "type": "Web"})
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None

web_docs = [doc for url in web_urls if (doc := crawl_web_url(url))]
print("=== ALL web_docs ===)")
print(len(web_docs))



Error crawling https://x.com/FidesInnova/: 400 Client Error: Bad Request for url: https://x.com/FidesInnova/
=== ALL web_docs ===)
21


In [16]:
# --- Step 4: GitHub Crawler ---
github_urls = [
    "https://github.com/TheArchitect2000/Fides-Innova-WiKi/",
    "https://github.com/TheArchitect2000/Blockchain-based-IoT-Server",
    "https://github.com/TheArchitect2000/ZKP-IoT-Arm-Siemens-IoT2050-C",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-c",
    "https://github.com/TheArchitect2000/Fides-Innova-Smart-Contract-Protocol",
    "https://github.com/TheArchitect2000/ZKP-Blockchain-Explorer",
    "https://github.com/TheArchitect2000/evm-server",
    "https://github.com/TheArchitect2000/New-IoT-Device-Integration",
    "https://github.com/TheArchitect2000/zkiot-riscv-qemu-rust"    
]

def crawl_github_url(url):
    
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
                    "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else "No Title"
        text = soup.get_text(separator='\n', strip=True)
        return Document(page_content=text, metadata={"source": url, "title": title, "type": "Web"})
    except Exception as e:
        print(f"Error crawling GitHub {url}: {e}")
        return None

github_docs = [doc for url in github_urls if (doc := crawl_github_url(url))]

print("=== ALL github_docs ===)")
print(len(github_docs))



=== ALL github_docs ===)
9


In [17]:
# --- Step 5: YouTube Channel Crawler ---

# Fides Innova YouTube Channel ID
CHANNEL_ID = "UCrrqGYx98H1dPdZsNb1i9-g"
CHANNEL_URL = f"https://www.youtube.com/channel/{CHANNEL_ID}"

def fetch_video_urls(channel_url: str):
    ydl_opts = {
        'ignoreerrors': True,
        'quiet': True,
        'extract_flat': True,  # Only metadata, not downloading
        'force_generic_extractor': False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=False)
        video_urls = []
        video_ids = []

        if 'entries' in result:
            for entry in result['entries']:
                if entry and 'id' in entry:
                    video_url = f"https://www.youtube.com/watch?v={entry['id']}"
                    video_urls.append(video_url)
                    video_ids.append(entry['id'])
        return video_ids

# Fetch and display video URLs
video_ids = fetch_video_urls(CHANNEL_URL)
print(f"✅ Found {len(video_ids)} videos on channel.\n")

youtube_docs = []
second_try_idx = []
third_try_idx = []
for idx in video_ids:
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded.")
    except:
        print(f"{idx} is not loaded.")
        second_try_idx.append(str(idx))
        print("\n unloaded list ")
        print(second_try_idx)
        print("\n")

print("\n")
print("second try for unloded videos.")

for idx in second_try_idx:
    print("waiting 10 seconds...")
    time.sleep(10)
    a = YoutubeLoader(idx)
    try:
        youtube_docs.extend(a.load())
        print(idx + " is loaded in the second try.")
    except:
        print("\n")
        print(f"{idx} is not loaded in the second try.")
        third_try_idx.append(str(idx))
        print("\n unloaded list for the second time ")
        print(second_try_idx)
        print("\n")

def change_YouTube_doc(doc):
    doc.metadata['type']='YouTube'
    return doc

youtube_docs = list(map(change_YouTube_doc, youtube_docs))

print("=== ALL youtube videos ===)")
print(len(youtube_docs))
print("=== Loaded youtube videos ===)")
print(len(youtube_docs)-len(third_try_idx))


✅ Found 29 videos on channel.

kgYxyxeDNl4 is loaded.
VZMBE2NLSC4 is loaded.
FtELZXp3qmQ is loaded.
loHb12HmJas is loaded.
YpfFHI3Ivmo is not loaded.

 unloaded list 
['YpfFHI3Ivmo']


sPWmDRItCNk is loaded.
1DAkWOP_uY4 is loaded.
2R_bDZ0sasM is loaded.
3GVfyu4uzhs is not loaded.

 unloaded list 
['YpfFHI3Ivmo', '3GVfyu4uzhs']


c8dQtxByOcY is loaded.
RcaMNALojIo is loaded.
qMjLWztjqvk is loaded.
DzvQhVw0tNU is not loaded.

 unloaded list 
['YpfFHI3Ivmo', '3GVfyu4uzhs', 'DzvQhVw0tNU']


asw_A0VSObI is loaded.
dBEEPLjEJGU is not loaded.

 unloaded list 
['YpfFHI3Ivmo', '3GVfyu4uzhs', 'DzvQhVw0tNU', 'dBEEPLjEJGU']


s1ZPMrqU0c0 is loaded.
K_7RhSkWgbM is loaded.
92QUILNAhDI is not loaded.

 unloaded list 
['YpfFHI3Ivmo', '3GVfyu4uzhs', 'DzvQhVw0tNU', 'dBEEPLjEJGU', '92QUILNAhDI']


WZnxV-f6E7k is not loaded.

 unloaded list 
['YpfFHI3Ivmo', '3GVfyu4uzhs', 'DzvQhVw0tNU', 'dBEEPLjEJGU', '92QUILNAhDI', 'WZnxV-f6E7k']


tNWEXETid7Q is loaded.
8Jk19GqcEW4 is not loaded.

 unloaded list 
['YpfF

In [18]:
# --- Step 6: Load PDFs ---
pdf_docs = []
pdf_files = [
    "PDF/zkIoT.pdf",
]

for path in pdf_files:
    try:
        loader = PyPDFLoader(path)
        pdf_docs.extend(loader.load())
        print(len(pdf_docs))
    except Exception as e:
        print(f"Error loading PDF {path}: {e}")

def change_pdf_doc(doc):
    doc.metadata['type']='PDF'
    return doc

pdf_docs = list(map(change_pdf_doc, pdf_docs))

print("=== ALL PDF docs ===)")
print(len(pdf_docs))

7
=== ALL PDF docs ===)
7


In [19]:

pptx_docs = []
pptx_files = [
    "PPTX/FidesinnovaDeck-v11.pptx"
]

for path in pptx_files:
    try:
        loader = UnstructuredPowerPointLoader(path)
        pptx_docs.extend(loader.load())
        print(f"✅ Loaded: {path}")
    except Exception as e:
        print(f"❌ Error loading PPTX {path}: {e}")

# Add metadata
def change_pptx_doc(doc):
    doc.metadata['type'] = 'PPTX'
    return doc

pptx_docs = list(map(change_pptx_doc, pptx_docs))

print("=== ALL PPTX docs ===)")
print(len(pptx_docs))

✅ Loaded: PPTX/FidesinnovaDeck-v11.pptx
=== ALL PPTX docs ===)
1


In [27]:
# --- Step 7: Split & Vectorize ---
all_docs = web_docs + github_docs + youtube_docs + pdf_docs + pptx_docs

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
split_docs = splitter.split_documents(all_docs)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    collection_name="fides_crawled_data",
    embedding_function=embedding,
    persist_directory="./chroma_langchain_db2"
)

vectordb.add_documents(split_docs)
# print("✅ All documents crawled, split, and stored in vector DB.")

# print("=== ALL Doc.s ===)")
# print(len(all_docs))

['edb43c62-1050-4267-a2ce-0f8998aea951',
 '876b7675-a982-45f8-8022-ab0446032876',
 'c6bf6b14-3a2e-4695-8f8b-93d7bf2cfff6',
 'fe983ed4-421f-4680-b792-ff0ad2907576',
 '9c65299e-4e4a-4bfc-a6c4-4aba251f6ac0',
 '3496aeef-aa4f-4bfe-b910-f94c9c650ae0',
 '67fff5c6-69f3-401c-8bff-30ac2a3dc2bd',
 'd4715735-6614-4c80-9ce4-8e00c9a22fab',
 '64818e45-6a58-4037-be9b-c63592efb420',
 '5cda0d57-48b5-4b6e-87c8-3433aac624aa',
 '5f97f93a-b7e8-4139-bb11-74686d5f3efb',
 '9dc52ce7-f288-4d94-864d-fc5008a963aa',
 '0bab70d5-1507-4ac3-9013-210e9649e7e5',
 'fd78f4c8-6417-4e44-9d19-2e2edd31cfed',
 '4999d6b1-9440-45ed-ad36-eadcaa7a5fb9',
 '76317140-9bda-454d-a6a8-882d13659aa1',
 '81aaea70-17d7-4500-b5ff-43135f8e9c94',
 '3ec40d02-3f97-4092-92c3-9b5f6c34e2a8',
 '49c42291-4f6c-4854-91f3-eca2e05afdc2',
 '9591e01f-e0fb-4ade-8ad3-70bf7264044c',
 'c4a3e690-65e5-4cca-947a-7d774fcf5446',
 '77781a98-bd13-4bc3-b91e-01217c787b2a',
 '8f50805c-7358-4ec7-80b6-5ab4866c7f57',
 'cc6e40cf-a9fd-4d76-bfd7-e79d65b27c53',
 'b77e0258-f5d5-

In [23]:
all_docs

[Document(metadata={'source': 'https://www.fidesinnova.io/', 'title': 'FidesInnova-Verified Computations for Reliable Results - FidesInnova', 'type': 'Web'}, page_content='FidesInnova-Verified Computations for Reliable Results - FidesInnova\nSkip to content\nSkip to footer\nProducts\nNetwork ZKP Explorer\nMobile App\nIoT Server\nzk-IoT ZKP SDK\nSample IoT Device\nEcosystem\nSensors (zkSensor)\nTransportation (MotionCertified)\nOil&Gas (EnergyWiseNetwork)\nMachine Learning (TrustLearn)\nSmart Home (TrustSense)\nPartnership\nResources\nWiKi\nGithub\nArticles\nAcademy\nzk-IoT Paper\nAbout Us\nContact Us\nAbout Us\nX-twitter\nYoutube\nGithub\nLink\nProducts\nNetwork ZKP Explorer\nMobile App\nIoT Server\nzk-IoT ZKP SDK\nSample IoT Device\nEcosystem\nSensors (zkSensor)\nTransportation (MotionCertified)\nOil&Gas (EnergyWiseNetwork)\nMachine Learning (TrustLearn)\nSmart Home (TrustSense)\nPartnership\nResources\nWiKi\nGithub\nArticles\nAcademy\nzk-IoT Paper\nAbout Us\nContact Us\nAbout Us\nX-t

In [ ]:
len(vectordb)